In [1]:
!pip install ultralytics
!apt-get install ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.2/699.2 kB 6.5 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [2]:
import os
from ultralytics import YOLO
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import numpy as np

In [3]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
ROOT_DIR = '/content/gdrive/MyDrive/fish_data'

In [5]:
model_path = os.path.join(ROOT_DIR, 'results_on_100_epochs', 'runs', 'detect', 'train', 'weights', 'best.pt')
model = YOLO(model_path)

In [ ]:
video_path = os.path.join(ROOT_DIR, 'videos', 'fish_video_1.mp4')
video_path_out = os.path.join(ROOT_DIR, 'output_video', 'fish_video_1_(only length).avi')

In [ ]:
no_of_fishes = []

cap = cv2.VideoCapture(video_path)

fps = int(cap.get(cv2.CAP_PROP_FPS)/2)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(video_path_out, fourcc, fps, (width, height))

while True:
  ret, frame = cap.read()

  if frame is None:
    break

  # detecting aruco marker
  parameters = cv2.aruco.DetectorParameters()
  aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_5X5_100)
  corners, _, _ = cv2.aruco.detectMarkers(frame, aruco_dict, parameters=parameters)

  int_corners = np.int0(corners)
  cv2.polylines(frame, int_corners, True, (225, 255, 0), 2)


  #pixel-to-cm conversion
  aruco_perimeter = cv2.arcLength(corners[0], True)
  pixel_cm_ratio = aruco_perimeter / 20


  # detecting fishes
  results = model(frame)[0]

  threshold = 0.2
  count = 0
  for result in results.boxes.data.tolist():
    x1, y1, x2, y2, score, class_id = result

    object_length = abs((x1-x2)/pixel_cm_ratio)
    object_width = abs((y1-y2)/pixel_cm_ratio)

    if score > threshold:

        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(frame, "Length {} cm".format(round(object_length, 1)), (int(x1), int(y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
        #cv2.putText(frame, "Width {} cm".format(round(object_width, 1)), (int(x1), int(y1 - 40)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)

        count = count + 1

  cv2.putText(frame, "Total No of Fishes are: " + str(count), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 1, cv2.LINE_AA)
  no_of_fishes.append(count)
  out.write(frame)
  ret, frame = cap.read()

cap.release()
out.release()
cv2.destroyAllWindows()


0: 384x640 3 fishs, 760.6ms
Speed: 6.5ms preprocess, 760.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 fishs, 671.4ms
Speed: 3.8ms preprocess, 671.4ms inference, 8.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 fishs, 467.2ms
Speed: 3.6ms preprocess, 467.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 fishs, 516.6ms
Speed: 5.7ms preprocess, 516.6ms inference, 5.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 fishs, 297.1ms
Speed: 7.7ms preprocess, 297.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 fish, 375.1ms
Speed: 5.7ms preprocess, 375.1ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 fish, 641.0ms
Speed: 4.2ms preprocess, 641.0ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 fishs, 505.9ms
Speed: 5.5ms preprocess, 505.9ms inference, 1.2ms postprocess per image at shape (1,